In [24]:
'''

investigate who gets oligodendroglioma (age,sex,etc), what treatment they have and how long they live

'''

import sqlite3
import pandas as pd
import numpy as np
import os

# Change the working directory to connect to the database
print(os.getcwd())
os.chdir('/data/master/DS175/share/0_Database')
print(os.getcwd())

# Connect to the database
conn = sqlite3.connect('Gliocova.db')
cur = conn.cursor()

def grouping(df,column):
    
    grouped_df = df.groupby(column)['PSEUDO_PATIENTID'].nunique()
    
    return grouped_df

# Patients count each year
query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
patients = pd.read_sql(query,conn)
patients = patients.drop_duplicates()
patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)

patients.dropna(axis = 0, how = 'any', subset = ['AGE', 'SEX', 'BIGTUMOURCOUNT', 'BASISOFDIAGNOSIS', 'GRADE'], inplace = False)
year_by_sex = patients.groupby(['SEX','DIAGNOSISYEAR'])['PSEUDO_PATIENTID'].count()
print(year_by_sex)
# total patient number: 1335

/data/master/DS175/share/0_Database
/data/master/DS175/share/0_Database
SEX  DIAGNOSISYEAR
1    2013              84
     2014             106
     2015             126
     2016             139
     2017             126
     2018             113
2    2013             101
     2014             109
     2015             101
     2016             111
     2017             102
     2018             117
Name: PSEUDO_PATIENTID, dtype: int64


In [6]:
# Basis of diagnosis
grouping(patients, 'BASISOFDIAGNOSIS')
# mostly were cutting and bone marrow biopsy (1255/1316 = 0.9536)

BASISOFDIAGNOSIS
1       4
2      52
7    1279
Name: PSEUDO_PATIENTID, dtype: int64

In [25]:
grouping(patients, 'INCOME_QUINTILE_2015')

INCOME_QUINTILE_2015
1 - Least deprived    286
2                     283
3                     277
4                     228
5 - Most deprived     261
Name: PSEUDO_PATIENTID, dtype: int64

In [18]:
patients['AGE'].describe()

count    1335.000000
mean       47.169288
std        14.403058
min        18.000000
25%        36.000000
50%        46.000000
75%        57.000000
max        99.000000
Name: AGE, dtype: float64

In [4]:
grouping(patients, 'ETHNICITY')

ETHNICITY
A    1061
B       8
C      95
D       3
F       3
G       2
H      30
J      22
L      29
M       8
N       5
P       2
R       5
S      36
X       7
Z      19
Name: PSEUDO_PATIENTID, dtype: int64

In [17]:
grouping(patients, 'DEATHLOCATIONDESC')

DEATHLOCATIONDESC
HOSPICE NOS      49
HOSPITAL         92
NURSING HOME     21
OTHER            10
PRIVATE HOME    108
UNKNOWN          34
Name: PSEUDO_PATIENTID, dtype: int64

In [5]:
# big tumours
grouping(patients, 'BIGTUMOURCOUNT')


BIGTUMOURCOUNT
1    1168
2     152
3      13
4       2
Name: PSEUDO_PATIENTID, dtype: int64

In [6]:
# ETHNICITY
grouping(patients, 'ETHNICITYNAME')

ETHNICITYNAME
ANY OTHER ASIAN BACKGROUND           29
ANY OTHER BLACK BACKGROUND            2
ANY OTHER ETHNIC GROUP               36
ANY OTHER MIXED BACKGROUND            2
ANY OTHER WHITE BACKGROUND           95
ASIAN INDIAN                         30
ASIAN PAKISTANI                      22
BLACK AFRICAN                         5
BLACK CARIBBEAN                       8
CHINESE                               5
MIXED WHITE AND ASIAN                 3
MIXED WHITE AND BLACK CARIBBEAN       3
NOT KNOWN                             7
NOT STATED                           19
WHITE BRITISH                      1061
WHITE IRISH                           8
Name: PSEUDO_PATIENTID, dtype: int64

In [7]:
# sex ratio
grouping(patients, 'SEX')


SEX
1    694
2    641
Name: PSEUDO_PATIENTID, dtype: int64

In [8]:
# VS by SEX
VS_S = patients.groupby(["VITALSTATUS", "SEX"])["PSEUDO_PATIENTID"].nunique()
print(VS_S)

VITALSTATUS  SEX
A            1      531
             2      482
D            1      163
             2      159
Name: PSEUDO_PATIENTID, dtype: int64


In [9]:
# VS by age
VS_a = patients.groupby(["VITALSTATUS", "AGE"])["PSEUDO_PATIENTID"].nunique()
print(VS_a.describe())

count    128.000000
mean      10.429688
std        9.526328
min        1.000000
25%        3.000000
50%        6.000000
75%       15.250000
max       38.000000
Name: PSEUDO_PATIENTID, dtype: float64


In [10]:
# Number of Big tumours
grouping(patients, 'BIGTUMOURCOUNT')

BIGTUMOURCOUNT
1    1168
2     152
3      13
4       2
Name: PSEUDO_PATIENTID, dtype: int64

In [11]:
# Where the tumour is
grouping(patients, 'SITE_CODED_DESC')


SITE_CODED_DESC
Brain, NOS                     128
Cerebellum, NOS                  1
Cerebrum                        48
Cranial nerve, NOS               1
Frontal lobe                   755
Occipital lobe                  23
Overlapping lesion of brain     34
Parietal lobe                  149
Spinal cord                      1
Temporal lobe                  195
Name: PSEUDO_PATIENTID, dtype: int64

In [3]:
# Age
import pandas as pd
import sys
import math
import seaborn as sns

query_male = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 1) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_male = pd.read_sql(query_male,conn)
df_male = df_male.drop_duplicates(subset=['PSEUDO_PATIENTID'])

query_female = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 2) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_female = pd.read_sql(query_female,conn)
df_female = df_female.drop_duplicates(subset=['PSEUDO_PATIENTID'])
df_male = df_male['AGE'].astype(int)
df_female = df_female['AGE'].astype(int)

data = {
'M': [df_male],
'F': [df_female],
'index' : ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
         '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100-104'],
}
       
pyramid_data = pd.DataFrame(data)
pyramid_data.columns

sns.set_style("white")
bar_plot = sns.barplot(y = 'index', x = 'M', color = 'blue', data = pyramid_data, order = ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100-104'])

bar_plot = sns.barplot(y = 'index', x = 'F', color = 'red', data = pyramid_data, order = ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100-104'])
bar_plot.set(xlabel = 'Age-Group', ylabel = 'Patients number', title = 'Patients Pyramid by sex')



ValueError: All arrays must be of the same length

In [ ]:
# test again
import plotly
from plotly.offline import plot
import plotly.graph_objs as go
import numpy as np

query_male = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 1) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_male = pd.read_sql(query_male,conn)
df_male = df_male.drop_duplicates(subset=['PSEUDO_PATIENTID'])

query_female = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 2) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_female = pd.read_sql(query_female,conn)
df_female = df_female.drop_duplicates(subset=['PSEUDO_PATIENTID'])

bins = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
labels = ['16_20', '21_25', '26_30', '31_35', '36_40', '41_45', '46_50', '51_55', 
          '56-60', '61-65', '66-70', '71_75', '76_80', '81_85', '86_90', '91_95', '96_100']

male = pd.DataFrame(pd.cut(df_male['AGE'], bins = bins, labels = labels))
female = pd.DataFrame(pd.cut(df_female['AGE'], bins = bins, labels = labels))

names = ['age']

male.columns = names
female.columns = names

male = male.groupby('age').count()
print(male)
female = female.groupby('age').count()


y = list(range(0, 100, 5))

layout = go.Layout(yaxis = go.layout.YAxis(title='Age'),
                  xaxis = go.layout.XAxis(
                      range = [100, -100],
                      tickvals = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0,
                                  10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                      ticktext = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                      title = 'Number'),
                  barmode = 'overlay',
                  bargap = 0.1)

data = [go.Bar(y = y,
           x = male,
           orientation = 'h',
           name = 'male_patients',
           hoverinfo = 'x', 
           marker = dict(color = 'sky blue')
           ),
       go.Bar(y = y,
           x = female,
           orientation = 'h',
           name = 'female_patients',
           hoverinfo = 'x', 
           marker = dict(color = 'orange')
           )]

plotly.offline.iplot(dict(data = data, layout = layout))


In [ ]:
# test with loc function
import plotly
from plotly.offline import plot
import plotly.graph_objs as go
import numpy as np

query_male = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 1) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_male = pd.read_sql(query_male,conn)
df_male = df_male.drop_duplicates(subset=['PSEUDO_PATIENTID'])

query_female = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 2) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_female = pd.read_sql(query_female,conn)
df_female = df_female.drop_duplicates(subset=['PSEUDO_PATIENTID'])

df_male['AGE'].astype(int)
df_female['AGE'].astype(int)

df_male.loc[(15 < df_male.AGE) & (df_male.AGE < 21), 'group'] = '16_20'
df_male.loc[(20 < df_male.AGE) & (df_male.AGE < 26), 'group'] = '21_25'
df_male.loc[(25 < df_male.AGE) & (df_male.AGE < 31), 'group'] = '26_30'
df_male.loc[(30 < df_male.AGE) & (df_male.AGE < 36), 'group'] = '31_35'
df_male.loc[(35 < df_male.AGE) & (df_male.AGE < 41), 'group'] = '36_40'
df_male.loc[(40 < df_male.AGE) & (df_male.AGE < 46), 'group'] = '41_45'
df_male.loc[(45 < df_male.AGE) & (df_male.AGE < 51), 'group'] = '46_50'
df_male.loc[(50 < df_male.AGE) & (df_male.AGE < 56), 'group'] = '51_55'
df_male.loc[(55 < df_male.AGE) & (df_male.AGE < 61), 'group'] = '56_60'
df_male.loc[(60 < df_male.AGE) & (df_male.AGE < 66), 'group'] = '61_65'
df_male.loc[(65 < df_male.AGE) & (df_male.AGE < 71), 'group'] = '66_70'
df_male.loc[(70 < df_male.AGE) & (df_male.AGE < 76), 'group'] = '71_75'
df_male.loc[(75 < df_male.AGE) & (df_male.AGE < 81), 'group'] = '76_80'
df_male.loc[(80 < df_male.AGE) & (df_male.AGE < 86), 'group'] = '81_85'
df_male.loc[(85 < df_male.AGE) & (df_male.AGE < 91), 'group'] = '86_90'

df_female.loc[(15 < df_female.AGE) & (df_female.AGE < 21), 'group'] = '16_20'
df_female.loc[(20 < df_female.AGE) & (df_female.AGE < 26), 'group'] = '21_25'
df_female.loc[(25 < df_female.AGE) & (df_female.AGE < 31), 'group'] = '26_30'
df_female.loc[(30 < df_female.AGE) & (df_female.AGE < 36), 'group'] = '31_35'
df_female.loc[(35 < df_female.AGE) & (df_female.AGE < 41), 'group'] = '36_40'
df_female.loc[(40 < df_female.AGE) & (df_female.AGE < 46), 'group'] = '41_45'
df_female.loc[(45 < df_female.AGE) & (df_female.AGE < 51), 'group'] = '46_50'
df_female.loc[(50 < df_female.AGE) & (df_female.AGE < 56), 'group'] = '51_55'
df_female.loc[(55 < df_female.AGE) & (df_female.AGE < 61), 'group'] = '56_60'
df_female.loc[(60 < df_female.AGE) & (df_female.AGE < 66), 'group'] = '61_65'
df_female.loc[(65 < df_female.AGE) & (df_female.AGE < 71), 'group'] = '66_70'
df_female.loc[(70 < df_female.AGE) & (df_female.AGE < 76), 'group'] = '71_75'
df_female.loc[(75 < df_female.AGE) & (df_female.AGE < 81), 'group'] = '76_80'
df_female.loc[(80 < df_female.AGE) & (df_female.AGE < 86), 'group'] = '81_85'
df_female.loc[(85 < df_female.AGE) & (df_female.AGE < 91), 'group'] = '86_90'
df_female.loc[(95 < df_female.AGE) & (df_female.AGE < 101), 'group'] = '96_100'

y = list(range(20, 100, 5))

layout = go.Layout(yaxis = go.layout.YAxis(title='Age'),
                  xaxis = go.layout.XAxis(
                      range = [100, -100],
                      tickvals = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0,
                                  10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                      ticktext = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                      title = 'Number'),
                  barmode = 'overlay',
                  bargap = 0.1)

print(df_male.groupby(['group'])['SEX'].count())   # ['SEX'] or actually any other columns, print(df.groupby(['group']).count()) and u will know
print(df_female.groupby(['group'])['SEX'].count())

data = [go.Bar(y = y,
           x = df_male.groupby(['group'])['SEX'].count(),
           orientation = 'h',
           name = 'male_patients',
           text = abs(df_male.groupby(['group'])['SEX'].count()),
           hoverinfo = 'text', 
           marker = dict(color = 'sky blue')
           ),
       go.Bar(y = y,
           x = -1*df_female.groupby(['group'])['SEX'].count(),
           orientation = 'h',
           name = 'female_patients',
           text = abs(df_female.groupby(['group'])['SEX'].count()),
           hoverinfo = 'text', 
           marker = dict(color = 'orange')
           )]

plotly.offline.iplot(dict(data = data, layout = layout))

In [14]:
# 10 year gap
import plotly
from plotly.offline import plot
import plotly.graph_objs as go
import numpy as np

query_male = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 1) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_male = pd.read_sql(query_male,conn)
df_male = df_male.drop_duplicates(subset=['PSEUDO_PATIENTID'])

query_female = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX = 2) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
df_female = pd.read_sql(query_female,conn)
df_female = df_female.drop_duplicates(subset=['PSEUDO_PATIENTID'])

df_male['AGE'].astype(int)
df_female['AGE'].astype(int)

df_male.loc[(10 < df_male.AGE) & (df_male.AGE < 21), 'group'] = '11_20'
df_male.loc[(20 < df_male.AGE) & (df_male.AGE < 31), 'group'] = '21_30'
df_male.loc[(30 < df_male.AGE) & (df_male.AGE < 41), 'group'] = '31_40'
df_male.loc[(40 < df_male.AGE) & (df_male.AGE < 51), 'group'] = '41_50'
df_male.loc[(50 < df_male.AGE) & (df_male.AGE < 61), 'group'] = '51_60'
df_male.loc[(60 < df_male.AGE) & (df_male.AGE < 71), 'group'] = '61_70'
df_male.loc[(70 < df_male.AGE) & (df_male.AGE < 81), 'group'] = '71_80'
df_male.loc[(80 < df_male.AGE) & (df_male.AGE < 91), 'group'] = '81_90'
df_male.loc[(90 < df_male.AGE) & (df_male.AGE < 101), 'group'] = '91_100'
df_male.loc[(100 < df_male.AGE) & (df_male.AGE < 111), 'group'] = '101_110'

df_female.loc[(10 < df_female.AGE) & (df_female.AGE < 21), 'group'] = '11_20'
df_female.loc[(20 < df_female.AGE) & (df_female.AGE < 31), 'group'] = '21_30'
df_female.loc[(30 < df_female.AGE) & (df_female.AGE < 41), 'group'] = '31_40'
df_female.loc[(40 < df_female.AGE) & (df_female.AGE < 51), 'group'] = '41_50'
df_female.loc[(50 < df_female.AGE) & (df_female.AGE < 61), 'group'] = '51_60'
df_female.loc[(60 < df_female.AGE) & (df_female.AGE < 71), 'group'] = '61_70'
df_female.loc[(70 < df_female.AGE) & (df_female.AGE < 81), 'group'] = '71_80'
df_female.loc[(80 < df_female.AGE) & (df_female.AGE < 91), 'group'] = '81_90'
df_female.loc[(90 < df_female.AGE) & (df_female.AGE < 101), 'group'] = '91_100'
df_female.loc[(100 < df_female.AGE) & (df_female.AGE < 111), 'group'] = '101_110'

# y = list(range(10, 100, 10))

y = ['11_20', '21_30', '31_40', '41_50', '51_60', '61_70', '71_80', '81_90', '91_100']

layout = go.Layout(yaxis = go.layout.YAxis(title='Age'),
                  xaxis = go.layout.XAxis(
                      range = [-200, 200],
                      tickvals = [-200, -180, -160, -140, -120, -100, -80, -60, -40, -20, 0,
                                  20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
                      ticktext = [200, 180, 160, 140, 120, 100, 80, 60, 40, 20, 0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
                      title = 'Number'),
                  barmode = 'overlay',
                  bargap = 0.1)

print(df_male.groupby(['group'])['SEX'].count())   # ['SEX'] or actually any other columns, print(df.groupby(['group']).count()) and u will know
print(df_female.groupby(['group'])['SEX'].count())

data = [go.Bar(y = y,
           x = df_male.groupby(['group'])['SEX'].count(),
           orientation = 'h',
           name = 'male_patients',
           text = abs(df_male.groupby(['group'])['SEX'].count()),
           hoverinfo = 'text', 
           marker = dict(color = 'sky blue')
           ),
       go.Bar(y = y,
           x = -1*df_female.groupby(['group'])['SEX'].count(),
           orientation = 'h',
           name = 'female_patients',
           text = abs(df_female.groupby(['group'])['SEX'].count()),
           hoverinfo = 'text', 
           marker = dict(color = 'orange')
           )]

plotly.offline.iplot(dict(data = data, layout = layout))

group
11_20      3
21_30     87
31_40    174
41_50    177
51_60    128
61_70     84
71_80     37
81_90      4
Name: SEX, dtype: int64
group
11_20       4
21_30      74
31_40     152
41_50     157
51_60     117
61_70      88
71_80      40
81_90       8
91_100      1
Name: SEX, dtype: int64


In [1]:
# a trial to make double barchart
df_male.loc[(15 < df_male.AGE) & (df_male.AGE < 21), 'group'] = '16_20'
df_male.loc[(20 < df_male.AGE) & (df_male.AGE < 26), 'group'] = '21_25'
df_male.loc[(25 < df_male.AGE) & (df_male.AGE < 31), 'group'] = '26_30'
df_male.loc[(30 < df_male.AGE) & (df_male.AGE < 36), 'group'] = '31_35'
df_male.loc[(35 < df_male.AGE) & (df_male.AGE < 41), 'group'] = '36_40'
df_male.loc[(40 < df_male.AGE) & (df_male.AGE < 46), 'group'] = '41_45'
df_male.loc[(45 < df_male.AGE) & (df_male.AGE < 51), 'group'] = '46_50'
df_male.loc[(50 < df_male.AGE) & (df_male.AGE < 56), 'group'] = '51_55'
df_male.loc[(55 < df_male.AGE) & (df_male.AGE < 61), 'group'] = '56_60'
df_male.loc[(60 < df_male.AGE) & (df_male.AGE < 66), 'group'] = '61_65'
df_male.loc[(65 < df_male.AGE) & (df_male.AGE < 71), 'group'] = '66_70'
df_male.loc[(70 < df_male.AGE) & (df_male.AGE < 76), 'group'] = '71_75'
df_male.loc[(75 < df_male.AGE) & (df_male.AGE < 81), 'group'] = '76_80'
df_male.loc[(80 < df_male.AGE) & (df_male.AGE < 86), 'group'] = '81_85'
df_male.loc[(85 < df_male.AGE) & (df_male.AGE < 91), 'group'] = '86_90'

df_female.loc[(15 < df_female.AGE) & (df_female.AGE < 21), 'group'] = '16_20'
df_female.loc[(20 < df_female.AGE) & (df_female.AGE < 26), 'group'] = '21_25'
df_female.loc[(25 < df_female.AGE) & (df_female.AGE < 31), 'group'] = '26_30'
df_female.loc[(30 < df_female.AGE) & (df_female.AGE < 36), 'group'] = '31_35'
df_female.loc[(35 < df_female.AGE) & (df_female.AGE < 41), 'group'] = '36_40'
df_female.loc[(40 < df_female.AGE) & (df_female.AGE < 46), 'group'] = '41_45'
df_female.loc[(45 < df_female.AGE) & (df_female.AGE < 51), 'group'] = '46_50'
df_female.loc[(50 < df_female.AGE) & (df_female.AGE < 56), 'group'] = '51_55'
df_female.loc[(55 < df_female.AGE) & (df_female.AGE < 61), 'group'] = '56_60'
df_female.loc[(60 < df_female.AGE) & (df_female.AGE < 66), 'group'] = '61_65'
df_female.loc[(65 < df_female.AGE) & (df_female.AGE < 71), 'group'] = '66_70'
df_female.loc[(70 < df_female.AGE) & (df_female.AGE < 76), 'group'] = '71_75'
df_female.loc[(75 < df_female.AGE) & (df_female.AGE < 81), 'group'] = '76_80'
df_female.loc[(80 < df_female.AGE) & (df_female.AGE < 86), 'group'] = '81_85'
df_female.loc[(85 < df_female.AGE) & (df_female.AGE < 91), 'group'] = '86_90'
df_female.loc[(95 < df_female.AGE) & (df_female.AGE < 101), 'group'] = '96_100'

plt.bar(np.arange(len(df_male.groupby('group')['SEX'].count()))-0.2,df_male.groupby('group')['SEX'].count(), color='orange', label = 'male_patients')
plt.bar(np.arange(len(df_female.groupby('group')['SEX'].count()))+0.2, df_male.groupby('group')['SEX'].count(), color='skyblue', label = 'femaile_patients')
plt.ylim(5, 35)
plt.title('5 years Age group')
plt.legend()
# due to the fact that there are different group numbers of male/female patients, 

NameError: name 'df_male' is not defined

In [12]:
# 5 YEAR death rate

a_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'A') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
a_patients = pd.read_sql(a_query,conn)
a_patients = a_patients.drop_duplicates()
a_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)

s_bin = [0, 365, 730, 1095, 1460, 1825, 10000]
s_label = ['<=365', '366-730', '731-1095', '1096-1460', '1461-1825', '>5yr']

Surv = pd.cut(a_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())


>5yr         269
1096-1460    199
731-1095     198
366-730      185
1461-1825    162
<=365          0
Name: INTERVAL_DIAG_TO_VS, dtype: int64


In [13]:
# 5 YEAR death rate

d_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'D') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
d_patients = pd.read_sql(d_query,conn)
d_patients = d_patients.drop_duplicates()
d_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)
s_bin = [0, 365, 730, 1095, 1460, 1825, 10000]
s_label = ['<=365', '366-730', '731-1095', '1096-1460', '1461-1825', '>5yr']

Surv = pd.cut(d_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())

# So 5 years' death rate is 22/(22+269)= 7.56%


<=365        121
366-730       73
731-1095      53
1096-1460     28
1461-1825     22
>5yr          22
Name: INTERVAL_DIAG_TO_VS, dtype: int64


In [9]:
# 3 YEAR death rate

A_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'D') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
d_patients = pd.read_sql(d_query,conn)
d_patients = d_patients.drop_duplicates()
d_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)
s_bin = [0, 365, 730, 1095, 10000]
s_label = ['<=365', '366-730', '731-1095', '>3yr']

Surv = pd.cut(d_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())

<=365       121
366-730      73
>3yr         72
731-1095     53
Name: INTERVAL_DIAG_TO_VS, dtype: int64


In [10]:
# 3 YEAR death rate

D_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'A') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
d_patients = pd.read_sql(d_query,conn)
d_patients = d_patients.drop_duplicates()
d_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)
s_bin = [0, 365, 730, 1095, 10000]
s_label = ['<=365', '366-730', '731-1095', '>3yr']

Surv = pd.cut(d_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())

# So 3 years' death rate is 72/(72+630)= 10.26%

>3yr        630
731-1095    198
366-730     185
<=365         0
Name: INTERVAL_DIAG_TO_VS, dtype: int64


In [16]:
# 1st YEAR death rate

FA_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'A') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
FA_patients = pd.read_sql(FA_query,conn)
FA_patients = FA_patients.drop_duplicates()
FA_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)
s_bin = [0, 365, 10000]
s_label = ['<=365', '>1yr']

Surv = pd.cut(FA_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())

>1yr     1013
<=365       0
Name: INTERVAL_DIAG_TO_VS, dtype: int64


In [15]:
# 1st YEAR death rate

FD_query = "SELECT * from TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS = 'D') AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4))"
FD_patients = pd.read_sql(FD_query,conn)
FD_patients = FD_patients.drop_duplicates()
FD_patients.replace(r'^\s*$', np.nan, regex = True, inplace = True)
s_bin = [0, 365, 10000]
s_label = ['<=365', '>1yr']

Surv = pd.cut(FD_patients["INTERVAL_DIAG_TO_VS"], bins = s_bin, labels = s_label)

print(Surv.value_counts())

# SO FIRST YEAR death rate = 

>1yr     198
<=365    121
Name: INTERVAL_DIAG_TO_VS, dtype: int64
